In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def fetch_dengue_info(start_year: int,
                      end_year: int,
                      municipio_ids: list[int],
                      base_url: str = "https://info.dengue.mat.br/api") -> pd.DataFrame:
    """
    Baixa séries semanais de dengue para uma lista de municípios via API InfoDengue.
    Parâmetros:
      start_year, end_year — intervalo de anos (ex: 2010, 2025)
      municipio_ids        — lista de códigos IBGE dos municípios
      base_url             — URL base da API InfoDengue
    Retorno:
      DataFrame com colunas: 
        municipio_id, semana_ano, semana_num, casos
    """
    all_dfs = []
    for m_id in municipio_ids:
        for year in range(start_year, end_year + 1):
            # Formata datas de início e fim do ano
            dt_start = f"{year}-01-01"
            dt_end   = f"{year}-12-31"
            resp = requests.get(
                f"{base_url}/casos/semanal",
                params={
                    "municipio": m_id,
                    "data_inicio": dt_start,
                    "data_fim": dt_end
                })
            resp.raise_for_status()
            js = resp.json()
            df = pd.json_normalize(js)
            df["municipio_id"] = m_id
            all_dfs.append(df[[
                "municipio_id",
                "ano_semana",    # string "YYYY-WW"
                "semana",        # número da semana
                "casos"          # casos confirmados
            ]])
    result = pd.concat(all_dfs, ignore_index=True)
    # Quebra "YYYY-WW" em col_year + col_week
    y, w = zip(*result["ano_semana"].str.split("-W"))
    result["year"] = result["municipio_id"].astype(str)  # placeholder
    result["year"], result["week"] = y, w
    result["year"] = result["year"].astype(int)
    result["week"] = result["week"].astype(int)
    return result[["municipio_id", "year", "week", "casos"]]

def fetch_inmet_climate(station_ids: list[str],
                        start: str,
                        end: str,
                        base_url: str = "https://apitempo.inmet.gov.br/estacao") -> pd.DataFrame:
    """
    Baixa dados diários de temperatura e precipitação de várias estações INMET
    e agrega para semanal.
    Parâmetros:
      station_ids — lista de códigos de estação INMET (ex: ["A001","A002"])
      start,end   — strings 'YYYY-MM-DD'
    Retorno:
      DataFrame com índice datetime semanal e colunas:
      ['station', 't_mean', 'precip_sum']
    """
    dfs = []
    for st in station_ids:
        resp = requests.get(f"{base_url}/{st}/dados",
                            params={"dataInicial": start, "dataFinal": end})
        resp.raise_for_status()
        raw = pd.DataFrame(resp.json())
        raw["datahora"] = pd.to_datetime(raw["datahora"])
        raw.set_index("datahora", inplace=True)
        # Seleciona colunas de interesse
        dfw = raw[["temp", "prec"]].rename(columns={"temp": "t", "prec": "p"})
        # Reamostra para semanal: temperatura média e precipitação total
        w = dfw.resample("W-MON").agg({"t": "mean", "p": "sum"})
        w["station"] = st
        dfs.append(w)
    all_climate = pd.concat(dfs).reset_index()
    # rename semanal
    all_climate["year"] = all_climate["datahora"].dt.isocalendar().year
    all_climate["week"] = all_climate["datahora"].dt.isocalendar().week
    all_climate = all_climate.rename(columns={"t":"t_mean","p":"precip_sum"})
    return all_climate[["station","year","week","t_mean","precip_sum"]]

def fetch_ibge_population_density(municipio_ids: list[int],
                                  year: int,
                                  base_url: str = "https://servicodados.ibge.gov.br/api") -> pd.DataFrame:
    """
    Busca densidade populacional (hab/km²) de municípios via IBGE SIDRA.
    Parâmetros:
      municipio_ids — lista de códigos IBGE
      year          — ano do censo/projeção
    Retorno:
      DataFrame com colunas ['municipio_id','pop_density']
    """
    # Exemplo de endpoint: série 6579 (população residente), variável 9324 (densidade)
    df_list = []
    for m in municipio_ids:
        url = (f"{base_url}/v3/agregados/6579/periodos/{year}/"
               f"variaveis/9324?localidades=N3[{m}]")
        resp = requests.get(url)
        resp.raise_for_status()
        data = resp.json()[0]["resultados"][0]["series"][0]["serie"]
        pop_density = float(data[str(year)])
        df_list.append({"municipio_id": m, "pop_density": pop_density})
    return pd.DataFrame(df_list)

# Exemplo de uso:
if __name__ == "__main__":
    municipios = [3106200, 3106309]         # códigos IBGE de exemplo
    anos = (2010, 2025)
    info = fetch_dengue_info(*anos, municipio_ids=municipios)
    
    # definir estações INMET e período
    inmet = fetch_inmet_climate(
        station_ids=["A001","A002"], 
        start="2010-01-01", 
        end="2025-12-31"
    )
    
    pop = fetch_ibge_population_density(municipios, year=2020)
    
    # Merge final: casos + clima + densidade
    df = (
        info
        .merge(inmet, on=["year","week"], how="left")
        .merge(pop, on="municipio_id", how="left")
    )
    print(df.head())

HTTPError: 404 Client Error: Not Found for url: https://info.dengue.mat.br/api/casos/semanal?municipio=3106200&data_inicio=2010-01-01&data_fim=2010-12-31